# Sentiment Analysis

# Data : 네이버 영화 리뷰

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data= pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_data= pd.read_table('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [ ]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
test_data.head()

In [ ]:
len(train_data), len(test_data)

(150000, 50000)

# Q1. 트레이닝 데이터를 트레이닝 셋과 벨리데이션 셋으로 분할하시오.

**규칙**
1. train : valid = 8 : 2
2. label이 골고루 분할되어 들어가도록 stratify하게 분할한다.
3. random state, seed 등은 2021로 통일
4. 변수명 :
    * x_train, x_valid
    * y_train, y_valid
5. test_data도, x와 y를 분리해둔다.
    * x_test, y_test

In [ ]:
####################
## Your Code here ##
####################





# Q2. x를 문장들이 담긴 리스트로 변환하시오.

ex> ['와 이 영화 재밌음', '크아앙 울부짖었다' ,... ]

In [ ]:
####################
## Your Code here ##
####################





# Q3. X를 tokenize하고, index의 sequence로 바꾸어 내시오.

**규칙**
1. 상위 40000개 단어만 사용하기로 한다.

In [ ]:
####################
## Your Code here ##
####################





# Q4. 문장의 길이를 통일하시오.

**규칙**
1. pad는 문장의 앞에 덧댈 것.
2. 문장 최대 길이는 40


In [ ]:
####################
## Your Code here ##
####################





# Q5. X에서, 0으로만 데이터를 제거하시오.

In [ ]:
####################
## Your Code here ##
####################





# Q6. 모델링 하시오.

**규칙**
* compile까지 마치시오.
* 모델 구조는 아래와 같다.
    1. Embedding layer : 임베딩 차원 192
    2. Conv1D : 필터수 128개, 윈도우 사이즈 5, swish
    3. Conv1D : 필터수 128개, 윈도우 사이즈 5, swish
    4. MaxPool1D : 필터사이즈 2
    5. Bidirectional layer :
        * 정방향 : LSTM, 48
        * 역방향 : LSTM, 48
    6. Bidirectional layer :
        * 정방향 : LSTM, 64
        * 역방향 : GRU, 64
    7. MaxPool1D : 필터사이즈 2
    7. Bidirectional layer :
        * 정방향 : GRU, 96
        * 역방향 : LSTM, 96
    8. Conv1D : 128개, 윈도우 사이즈 6, swish
    9. Flatten
    10. Fully connected, 노드 1024개, swish
    10. 적절한 아웃풋레이어.

In [ ]:
####################
## Your Code here ##
####################





In [ ]:
####################
## Your Code here ##
####################





# Q7. Early Stopping을 이용하여 모델을 학습시키시오.

**규칙**
1. 트레이닝셋의 15%는 validation set으로 둔다.
2. batch size는 128 ~ 512 사이로 결정한다.
3. 10000 epochs
2. 10 epochs 이내에 개선이 안되면 stopping
3. 가장 성능이 좋았을 때의 가중치로 복원할 것


In [ ]:
####################
## Your Code here ##
####################





# Q8. 테스트 셋위에서 평가를 하시오.

0. positive : 1, negative : 0
1. positive로 간주하는 기준을 [ 5%, 10%, ... , 95% ] 19개로 가져간다.
    * 예시) 두번째 기준은 positive할 확률이 10%만 넘어가도 postive하다고 간주한다.
2. 각 19개 기준에 대하여, model의 sensitivity 변화를 출력한다.
    * optional) 시각화를 하여도 좋다.
3. 각 19개 기준에 대하여, model의 'positive' class의 precision 변화를 출력한다.
    * optional) 시각화를 하여도 좋다.


In [ ]:
####################
## Your Code here ##
####################





# Q9. 다음 텍스트가 positive할 확률을 출력하시오.

"이 영화는 명작인지, 망작인지 너무나 혼란스럽고 즐겁다."




In [ ]:
####################
## Your Code here ##
####################





# Q10. 아래 주어진 문장을, 주어진 조건에 따라 분석하시오.

review = '명작이지ㅋ 특히 명절때만 되면 티비에서 참 많이 해줬었는데, 웃기기도하고, 액션도 가미되었고...특히 가족같은 형제라는 주제가 안방에 잘 먹혔던거 같다.'

* 루프문과 print를 이용하여 아래와 같은 방식으로 출력하시오.
    1. positive확률 :  50%,  인풋 : '명작이지ㅋ'
    2. positive확률 :  55%, 인풋 : '명작이지ㅋ 특히'
    3. positive확률 : 56%, 인풋 : '명작이지ㅋ 특히 명절때만'
    4. .....(계속 한 단어씩 증가 시키기)
    5. positive확률 : 98%, 인풋 : '명작이지~ 같다.'
* 인풋으로 사용한 문장과 확률은 명확히 출력되어야 한다.
    * 나머지는 출력되지 않아도 좋다.

In [ ]:
####################
## Your code here ##
####################




